In [1]:
import warnings
warnings.filterwarnings('ignore')
import time
import pandas as pd
from tqdm import tqdm 
pd.options.display.max_columns = 100

__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE

In [2]:
train = pd.read_pickle('../../data/ridge/train_xgb_baseline.pkl')
test = pd.read_pickle('../../data/ridge/test_xgb_baseline.pkl')

In [3]:
not_feats = ['instance_id', 'click', 'time_day', 'time']
cat_cols = test.columns
cat_cols = [col for col in cat_cols if 'count' not in col and col not in not_feats]

num_cols = [col for col in test.columns if 'count' in col]
start_feats = list(set(test.columns) - set(not_feats))

In [4]:
append_count = pd.read_pickle(FILE.X_fe_agg_count_formater.value.format(1))
append_timeNext = pd.read_pickle(FILE.X_fe_agg_time_next_formater.value.format(1))
append_timeCount = pd.read_pickle(FILE.X_fe_agg_time_count_formater.value.format(1))

X_fe = append_count.merge(append_timeNext,on='instance_id',how='inner')
X_fe = X_fe.merge(append_timeCount,on='instance_id',how='inner')
print(X_fe.shape)

# fillna
cols = list(set(X_fe.columns)-set(['instance_id']))
for col in tqdm(cols):
    if 'to_n_previous' in col:
        X_fe[col] = X_fe[col].fillna(1)
    elif 'to_n_next' in col:
        X_fe[col] = X_fe[col].fillna(-1)
    else:
        X_fe[col] = X_fe[col].fillna(0)


 10%|▉         | 31/319 [00:00<00:00, 309.55it/s]

(1041674, 320)


100%|██████████| 319/319 [00:01<00:00, 307.66it/s]


In [5]:
train = train.merge(X_fe,how='inner',on='instance_id')
print(train.shape)

test = test.merge(X_fe,how='inner',on='instance_id')
print(test.shape)


(1001650, 390)
(40024, 390)


In [6]:
all_feat = list(set(test.columns) - set(not_feats))

In [7]:
# import copy
# train_count = copy.copy(train)
# test_count = copy.copy(test)
# train_count = train_count.fillna(-99999)
# test_count = test_count.fillna(-99999)
# train_count["test"]=0
# test_count["test"]=1
# temp_df = pd.concat([train_count,test_count],axis=0)
# for col in tqdm(cat_cols):
#     temp_df.loc[:,col]=temp_df.loc[:,col].astype('category')
# train_count=temp_df[temp_df["test"]==0].iloc[:,:-1]
# test_count=temp_df[temp_df["test"]==1].iloc[:,:-1]

# for col in tqdm(cat_cols):
# #     counts = train_count.iloc[:,i].value_counts()
#     counts = train_count[col].value_counts()
#     counts = counts.sort_index()
#     counts = counts.fillna(0)
#     counts += np.random.rand(len(counts))/1000
#     train_count.loc[:,col].cat.categories = counts
#     test_count.loc[:,col].cat.categories = counts
    
# for col in cat_cols:
#     train_count[col] = train_count[col].astype(float)
#     test_count[col] = test_count[col].astype(float)
    
# train_count[num_cols] = train_count[num_cols].fillna(0)

In [8]:
train_p = train
X = train_p.drop(columns=not_feats)
y = train_p['click']

In [9]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score
clf = LogisticRegression ()

In [10]:

# start_feats = ['model', 'user_id', 'count_model', 'count_city', 'amt_usertags', 'osv', 'inner_slot_id']
feats_to_test = [feat for feat in all_feat if feat not in start_feats]

In [11]:
init_feats = start_feats
init_score = cross_val_score(clf, X[init_feats], y, cv=5, scoring='neg_log_loss')
init_score = init_score.mean()
print('init auc: ', init_score)

for i, feat in enumerate(feats_to_test):
    scores = cross_val_score(clf, X[init_feats+[feat]], y, cv=5, scoring='neg_log_loss')
    auc = scores.mean()
    if auc > init_score:
        init_score = auc
        init_feats = init_feats+[feat]
        print('#{}/{} | YES! {} [{}] {:.6f}'.format(i, len(feats_to_test), feat, len(init_feats), init_score))
    else:
        print('#{}/{} | NO!! {} [{}] {:.6f}'.format(i, len(feats_to_test), feat, len(init_feats), init_score))
    time.sleep(1)

init auc:  -0.4345769363054794
#0/319 | NO!! osv_count_in_previous_n_time_unit_6hour [67] -0.434577
#1/319 | YES! inner_slot_id_count_in_previous_n_time_unit_6hour [68] -0.434519
#2/319 | NO!! user_tags_count_in_next_n_time_unit_6hour [68] -0.434519
#3/319 | NO!! province_reverse_cumulative_count [68] -0.434519
#4/319 | NO!! creative_is_jump_time_to_n_previous [68] -0.434519
#5/319 | NO!! creative_is_jump_count_in_previous_n_time_unit_1hour [68] -0.434519
#6/319 | NO!! city_count_in_previous_n_time_unit_1hour [68] -0.434519
#7/319 | NO!! creative_has_deeplink_count_in_previous_n_time_unit_1hour [68] -0.434519
#8/319 | YES! make_cumulative_count [69] -0.434498
#9/319 | NO!! adid_count_in_next_n_time_unit_1hour [69] -0.434498
#10/319 | YES! creative_type_count_in_previous_n_time_unit_6hour [70] -0.434441
#11/319 | NO!! city_count_in_next_n_time_unit_1hour [70] -0.434441
#12/319 | NO!! model_cumulative_count [70] -0.434441
#13/319 | NO!! os_count_in_previous_n_time_unit_6hour [70] -0.4344

KeyboardInterrupt: 

In [ ]:
# you have selected:
len(init_feats)

In [ ]:
not_selected = set(feats) - set(init_feats)
not_selected